In [2]:
import networkx as nx
import random
random.seed(0)
import numpy as np
np.random.seed(0)

In [3]:
G = nx.erdos_renyi_graph(10, 0.3, seed = 1, directed = False)
G

### Defining the `next_node` function with the list of our parameters:

In [4]:
def next_node(previous, current, p, q):
    # retrieving the list of neighboring nodes
    # from the current node and initialise the list
    # of alpha values
    neighbors = list(G.neighbors(current))
    alphas = []

    # for each neighbor, calculate appropriate alpha value ie,
    # 1/p -> if neighbor is previous node,
    # 1 -> if neighbor is connected to previous node,
    # 1/q -> otherwise
    for neighbor in neighbors:
        if neighbor == previous:
            alpha = 1/p
        elif G.has_edge(neighbor, previous):
            alpha = 1
        else:
            alpha = 1/q
        
        alphas.append(alpha)

    # now we normalise these values to create probabilities
    probs = [alpha / sum(alphas) for alpha in alphas]

    # now we randomly select the next node based on the transition
    # probabilities calculated in the previous step
    next = np.random.choice(neighbors, size=1, p = probs)[0]

    return next

before this function can be tested, we need the code to generate the entire random walk. <br>

the next node is chosen by the `next_node()`, which requires additional parameter: `p` and `q`, but also the previous and the current nodes. <br>

These nodes can be easily obtained by looking at the two last elements added to the `walk` variable. We also return strings instead of integers for compatibility reasons

In [5]:
# updated version of the random_walk() method
def random_walk(start, length, p, q):
    walk = [start]

    for i in range(length):
        current = walk[-1]
        previous = walk[-2] if len(walk) > 1 else None
        next = next_node(previous, current, p, q)
        walk.append(next)
    
    return [str(x) for x in walk]

In [6]:
random_walk(0, 8, p = 1, q = 1)

['0', '4', '7', '6', '4', '5', '4', '5', '6']

now, let's bias them toward going back to the previous node with `q = 10`

In [7]:
random_walk(0, 8, p = 1, q = 10)

['0', '9', '1', '9', '1', '9', '1', '0', '1']

This time, the random walk explores more nodes in the graph. You can see that it never goes back to the previous node because the probability is low with `p = 10`:

In [8]:
random_walk(0, 8, p = 10, q = 1)

['0', '1', '9', '4', '7', '8', '7', '4', '6']

In [9]:
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [10]:
# loading the dataset - Zachary's Karate club
G = nx.karate_club_graph()

# transforming the nodes' labels into numerical values (0 and 1):
labels = []
for node in G.nodes:
    label = G.nodes[node]['club']
    labels.append(1 if label == 'Officer' else 0)

Now we generate a list of random walks as seen previously using our `random_walk()` method 80 times for each node in the graph.

In [11]:
walks = []

for node in G.nodes:
    for _ in range(80):
        walks.append(random_walk(node, 10, 3, 2))

### Creating an instance of Word2Vec (a skip gram model) with a hierarchical `softmax` function:

In [12]:
node2vec = Word2Vec(walks,
                    hs = 1, # hierarchical softmax
                    sg = 1, # skip-gram,
                    vector_size = 100,
                    window = 10,
                    workers = 2,
                    min_count = 1,
                    seed = 0)

The skip gram model is now trained on the sequences we generated for 30 epochs:

In [13]:
node2vec.train(walks, total_examples = node2vec.corpus_count, epochs = 30, report_delay = 1)

(185807, 897600)

we now craete masks to train and test the classifier:

In [14]:
train_mask = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
train_mask_str = [str(x) for x in train_mask]
test_mask = [0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21,
23, 25, 26, 27, 28, 29, 30, 31, 32, 33]
test_mask_str = [str(x) for x in test_mask]
labels = np.array(labels)

The random forest classifier is trained on the training data:

In [15]:
clf = RandomForestClassifier(random_state = 0)
clf.fit(node2vec.wv[train_mask_str], labels[train_mask]) 

RandomForestClassifier(random_state=0)

We now evaluate it in terms of accuracy for the test data:

In [16]:
y_pred = clf.predict(node2vec.wv[test_mask_str])
acc = accuracy_score(y_pred, labels[test_mask])
print(f"Node2Vec accuracy = {acc * 100:.2f}%")

Node2Vec accuracy = 90.91%


# Building a movie `RecSys`

One of the most popular applications of GNNs is Recommendation System. If you think about it, the goal is to produce vectors (in this case, the name of the movies) with the ability to measure their similarity.

Ideally we would want to create [biased] random walks of movies, which requires a graph dataset where similar movies are connected to each other. This is not easy to find.

<i>Here, we will try to implement a simple and intuitive approach: movies that are liked by the same users are connected. We will then use this graph to learn movie embeddings using Node2Vec.</i>

In [17]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

url = 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

with urlopen(url) as zurl:
    with ZipFile(BytesIO(zurl.read())) as zfile:
        zfile.extractall('.')

we are interested in two files: `ratings.csv` and `movies.csv`. The first one stores all the rating made by users, and the second one allows us to translate movie identifiers into titles.

In [19]:
import pandas as pd

ratings = pd.read_csv('ml-100k/u.data', sep = '\t', names = ['user_id', 'movie_id', 'rating', 'unix_timestamp'])
ratings

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [20]:
# importing the movie dataset
movies = pd.read_csv('ml-100k/u.item', sep = '|', usecols = range(2), names = ['movie_id', 'title'], encoding = 'latin-1')
movies

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


Here, we would want to see all movies liked by the same user. This means that ratings such as 1, 2, and 3 and not very relevant. So, we only keep ratings 4 and 5.

In [21]:
ratings = ratings[ratings['rating'] >= 4]
ratings

,user_id,movie_id,rating,unix_timestamp
5,298,474,4,884182806
7,253,465,5,891628467
11,286,1014,5,879781125
12,200,222,5,876042340
16,122,387,5,879270459
...,...,...,...,...
99988,421,498,4,892241344
99989,495,1091,4,888637503
99990,806,421,4,882388897
99991,676,538,4,892685437


In [22]:
print(f"Number of unique users with 4 and 5 ratings: {ratings['user_id'].nunique()}")

Number of unique users with 4 and 5 ratings: 942


The next step is to count every time that two movies are liked by the same user.

In [23]:
from collections import defaultdict

pairs = defaultdict(int)

# loop through the entire list of users
for group in ratings.groupby('user_id'):
    # list of IDs of movies rated by the current user
    user_movies = list(group[1]["movie_id"])

    # count every time two movies are seen together
    for i in range(len(user_movies)):
        for j in range(i + 1, len(user_movies)):
            pairs[(user_movies[i], user_movies[j])] += 1

The `pairs` object now stores the number of times two movies have been liked by the same user. We can use this information to build the edges of our graph as follows:

In [24]:
G = nx.Graph()

# for each pair, we unpack the two movies and their corresponding score
for pair in pairs:
    movie1, movie2 = pair
    score = pairs[pair]

    # the edge is only created when the score is high enough
    if score >= 20:
        G.add_edge(movie1, movie2, weight = score)

print(f"Total number of graph nodes: {G.number_of_nodes()}")
print(f"Total number of graph edges: {G.number_of_edges()}")

Total number of graph nodes: 410
Total number of graph edges: 14936


In [26]:
from node2vec import Node2Vec

node2vec = Node2Vec(G, dimensions = 64, walk_length = 20, num_walks = 200, p = 2, q = 1, workers = 1)

/home/consumer/Desktop/advanced-gnn/gnn_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating walks (CPU: 1): 100%|██████████| 200/200 [00:19<00:00, 10.45it/s]


we now train a model on these biased random walks with a window of 10 (5 nodes before, 5 nodes after)

In [27]:
model = node2vec.fit(window = 10, min_count = 1, batch_words = 4)

In [28]:
def recommend(movie):
    """method to recommend movies based on a given title"""
    movie_id = str(movies[movies['title'] == movie].movie_id.values[0])
    for id in model.wv.most_similar(movie_id)[: 5]:
        title = movies[movies['movie_id'] == int(id[0])].title.values[0]
        print(f"{title}: {id[1]:.2f}")

recommend("Star Wars (1977)")

Return of the Jedi (1983): 0.61
Raiders of the Lost Ark (1981): 0.55
Godfather, The (1972): 0.49
Indiana Jones and the Last Crusade (1989): 0.46
White Squall (1996): 0.44


The model tells us that Return of the Jedi and Raiders of the Lost Ark are the most similar to Star Wars, although with a relatively low score (`< 0.7`). Nonetheless, this is a good result for our first step into the RecSys world! In later chapters, we’ll see more powerful models and approaches to building state-of-the-art RecSys.